# Column generation VRPTW

In [29]:
import os
dir_notebooks = os.getcwd()
dir_home, _ = os.path.split(dir_notebooks)

dir_data = os.path.join(dir_home, 'data')
dir_code = os.path.join(dir_home, 'cvrp')

dir_instances = os.path.join(dir_data, 'instances')
dir_solutions = os.path.join(dir_data, 'solutions')
dir_my_solutions = os.path.join(dir_data, 'my_solutions')

import vrplib # https://github.com/leonlan/VRPLIB
import networkx as nx  
import numpy as np

# Import my modules
import sys
sys.path.append(f"{dir_home}") 
sys.path.append(f"{dir_code}")

from cvrp.mp_cvrp import MP_CVRP #Implementar MP_VRPTW
from utils.utils import save_solution
import pylgrim 

In [30]:
# Read Solomon formatted instances
instance = vrplib.read_instance(f"{dir_instances}/R101.txt", instance_format="solomon")
solution = vrplib.read_solution(f"{dir_solutions}/R101.sol") # only 1 solution format

In [31]:
instance["node_coord"] = instance["node_coord"]
instance["capacity"] = 100

In [32]:
instance.keys()

dict_keys(['name', 'vehicles', 'capacity', 'node_coord', 'demand', 'time_window', 'service_time', 'edge_weight'])

In [33]:
name = instance["name"]
n_nodes = len(instance["node_coord"])

nodes = list(range(n_nodes))
customers = nodes[1:] # 0 is the depot
depot = 0
n_vehicles = 5 # Por la instancia

arcs = [(i,j) for i in nodes for j in nodes if i!=j]

In [34]:
mp_vrptw = MP_CVRP(name, customers, n_vehicles)

In [35]:
mp_vrptw.solve(time_limit = 60*60)
dual_customers, dual_depot = mp_vrptw.get_duals()

In [36]:
# Process duals
reduced_costs = {(i,j) : 0 for i,j in arcs}
for i,j in arcs:
    if j != depot:
        reduced_costs[i,j] = instance["edge_weight"][i,j] - dual_customers.get(j,0)
    else:
        reduced_costs[i,j] = instance["edge_weight"][i,j] - dual_depot

In [37]:
# Create graph
G = nx.DiGraph(n_res=2) # Capacity, time
for i in nodes:
    G.add_node(i, time_window = instance["time_window"][i], demand = instance["demand"][i])

    G.nodes[i]["res_min"] = np.array([0, instance["time_window"][i][0] ])
    G.nodes[i]["res_max"] = np.array([instance["capacity"], instance["time_window"][i][1] ])

for i,j in arcs:
    ij_demand = instance["demand"][j]
    ij_time = instance["edge_weight"][i,j]+instance["service_time"][i] # Tiempo de servicio

    G.add_edge(i,j, weight=reduced_costs[i,j], res_cost=np.array([ij_demand, ij_time]))

In [38]:
source = 0
print('Testing with {} nodes'.format(len(G)))

Testing with 101 nodes


In [39]:
from pylgrim import tools
source_in = 'source_in'
tools.decouple_source(G, source, source_in=source_in)

100

In [40]:
from pylgrim import ESPPRCTW
target = source_in
max_res = list([instance["capacity"], instance["time_window"][source][1]])
G_pre, res_min = ESPPRCTW.preprocess(G, source, target, max_res, res_name='res_cost')
shortest_path, shortest_path_label = ESPPRCTW.GSSA(G_pre, source, target, max_res, res_min, res_name='res_cost')


print('shortest path found: {} with label {}'.format(shortest_path, shortest_path_label))
print('')

while True:
    try:
        e = shortest_path.__next__()
        print('{} ⇨ {} : {}'.format(*e))
        print('')
    except StopIteration:
        # last element reached
        break

KeyboardInterrupt: 